## Layer of basic Operation
#### $
\begin{align}
\frac{\partial(x+y)}{\partial x} &=& 1 \\
\frac{\partial(x+y)}{\partial y} &=& 1 \\
\frac{\partial(x-y)}{\partial x} &=& 1 \\
\frac{\partial(x-y)}{\partial y} &=& -1 \\
\frac{\partial(xy)}{\partial x} &=& y \\
\frac{\partial(xy)}{\partial y} &=& x \\
\frac{\partial(x/y)}{\partial x} &=& 1/y \\
\frac{\partial(x/y)}{\partial y} &=& -x/y^2
\end{align}
$

# Backward method use to find Derivative

In [8]:
class Plus:
    def forward(self, x, y):
        return x + y
    def backward(self, g):
        return g, g
class Minus:
    def forward(self, x, y):
        return x - y
    def backward(self, g):
        return g, -g 
class Multiply:
    def forward(self, x, y):
        self.x = x
        self.y = y
        return x * y
    def backward(self, g):
        return g * self.y, g * self.x

class Divind:
    def forward(self, x, y):
        self.x = x
        self.y = y
        return x / y
    def backward(self, g):
        return (g/self.y),  (- g * self.y / self.y ** 2)


# Example Equation
# $$
\begin{align}
x = \frac{cd}{a-b}
\end{align}
$$
![EQ](https://phyblas.hinaboshi.com/rup/nayuki/umaki/i01.png)
##### given : a=1,b=2,c=3,d=4 

In [10]:
minus = Minus()
multiply = Multiply()
divind = Divind()

a, b, c, d = 1, 2, 3, 4
e = minus.forward(a, b)
f = multiply.forward(c, d)
x = divind.forward(f, e)
print('e = %d, f = %d, x = %d' % (e, f, x))

e = -1, f = 12, x = -12


In [12]:
gf, ge = divind.backward(1)
gc, gd = multiply.backward(gf)
ga, gb = minus.backward(ge)
print('ga=%d, gb=%d, gc=%d \ngd=%d, ge=%d, gf=%d' % (ga,gb,gc,gd,ge,gf))

ga=1, gb=-1, gc=-4 
gd=-3, ge=1, gf=-1


In [14]:
divind.backward(1)

(-1.0, 1.0)

## $g$ is derivative of that varaible
# Derivative of higher operation

In [15]:
import numpy as np
class Exp:
    def forward(self, x):
        self.expx = np.exp(x)
        return self.expx
    def backward(self, g):
        return g * self.expx
class Ln:
    def forward(self, x):
        self.x = x
        return np.log(x)
    def backward(self, g):
        return g / self.x


# Layers of ***Activation Functions***

In [16]:
class Sigmoid:
    def forward(self, x):
        self.h = 1/(1+np.exp(-x))
        return self.h
    def backward(self, h):
        return g * (1.0 - self.h) * self.h

class ReLu:
    def forward(self, x):
        self.filter = x > 0
        return np.where(self.filter, x, 0) # np.where(cond, if true, if false)
    def backward(self, g):
        return np.where(self.filter, g, 0)
    

# Layer with ***Adjustable Parameters***
#### Simple function with easy process

In [17]:
class MultiplyW_PlusB:
    def __init__(self, w, b):
        self.w = w
        self.b = b
        self.gw, self.gb = 0, 0
    def forward(self, x):
        self.x = x
        return self.w * x + self.b
    def backward(self, g):
        self.gw += g * self.x
        self.gb += g
        return g * self.w
    

# Using 

In [30]:
f1 = MultiplyW_PlusB(w=2, b=1)
f2 = MultiplyW_PlusB(3, 4)
x = 3
y = f1.forward(x)
print(y) # 7 = 2*3 + 1

7


In [31]:
z = f2.forward(x)
print(z) # 13 = 3*3 + 4

13


In [32]:
gy = f2.backward(1) 
print(gy) # 1 * 3 = g * self.w

3


In [33]:
gx = f1.backward(gy) 
gx # => g * self.w => gy * 2 => 3 * 2 => 6

6

In [35]:
print(f1.gw, f1.gb)

9 3


# Write in Matrix form
### Layers with linear computation are called **Affine Layer**
keras names "Dense layer", pytorch names "Linear layer"

In [36]:
class Affine:
    def __init__(self, w, b):
        self.w ,self.b = w, b
        self.gw, self.gb = 0, 0
    def forward(self, X):
        self.X = X
        return np.dot(X, self.w) + self.b
    def backward(self, g):
        self.gw += np.dot(self.X.T, g)
        self.gb += g.sum(0)
        return np.dot(g, self.w.T)

# Using

In [39]:
af = Affine(np.random.randint(0, 9, [3,4]),
           np.random.randint(0,9 ,4))
x = np.random.randint(0, 9, [2,3])
print(x)

[[7 1 7]
 [5 5 2]]


In [41]:
print(af.w)

[[4 4 7 8]
 [6 3 7 2]
 [5 4 2 3]]


In [43]:
print(af.b)

[3 6 8 6]


In [45]:
a = af.forward(x)
a

array([[72, 65, 78, 85],
       [63, 49, 82, 62]])

In [47]:
gx = af.backward(np.ones([2,4]))
gx

array([[23., 18., 14.],
       [23., 18., 14.]])

In [48]:
af.gw

array([[24., 24., 24., 24.],
       [12., 12., 12., 12.],
       [18., 18., 18., 18.]])

In [49]:
af.gb

array([4., 4., 4., 4.])